# Exemple d'analyse d'un dataset : le Titanic

**Exemple de données qualitatives**

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
titanic = pd.read_csv("../input/titanic.csv")

In [ ]:
titanic.head()

## Interprétation des paramètres

- survived - Survival (0 = No; 1 = Yes)
- pclass - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
- name - Name
- sex - Sex
- age - Age
- sibsp - Number of Siblings/Spouses Aboard
- parch - Number of Parents/Children Aboard
- ticket - Ticket Number
- fare - Passenger Fare
- cabin - Cabin
- embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)


In [ ]:
# Suppression des informations non pertinentes
t = titanic.drop(['name','ticket'], axis=1)
t.head()

# Rose & Jack

*value_counts* permet de compter le nombre d'éléments par catégorie d'une série

In [ ]:
t.sex.value_counts()      # nombre d'hommes et de femmes

In [ ]:
t.sex.count()              # nombre total hommes+femmes

In [ ]:
t.cabin.count()

In [ ]:
t.count()                  # Comptage par colonnes

On remarque qu'il manque des valeurs pour 'age' et 'embarked' (présence de valeurs indéfinies 'NaN')

On peut définir un booléen pour abréger une caractéristique :

In [ ]:
hommes = (t.sex=="male")

In [ ]:
t[hommes].head()        # t[hommes] est le tableau où on ne retient que lignes pour lesquelles hommes est True

On peut compter les hommes survivants ou non :

In [ ]:
t[hommes].survived.value_counts()

## Exercice : quelle est la probabilité de survie de Rose et Jack ?

<img src="https://www.scienceabc.com/wp-content/uploads/2016/04/titanic-jack-and-rose-plank-scene.webp">

In [ ]:
femmes = t.sex=="female"
classe1 = t.pclass == 1
classe2 = t.pclass == 2
classe3 = t.pclass == 3
survivant = t.survived == 1
mort = ~ survivant

Jack est un homme en 3ème classe, et Rose une femme en 1ère :

In [ ]:
jack = hommes & classe3
rose = femmes & classe1

In [ ]:
p_jack = t[jack & survivant].sex.count()/float(t[jack].sex.count())
print(p_jack)

In [ ]:
p_rose = t[rose & survivant].sex.count()/float(t[rose].sex.count())
print(p_rose)

## Enrichissement du dataset

l'opérateur *map* permet de transformer les valeurs d'une série. Par exemple, si on veut donner des valeurs explicites à la classe, on peut créer une **nouvelle colonne** *classe* :

In [ ]:
t['classe'] = t.pclass.map({1: "Premiere", 2: "Seconde", 3: "Troisieme"})

In [ ]:
t.head()

On peut aussi créer une colonne qui indique si un passager voyage *seul* ou non :

In [ ]:
t['seul'] = ~(t.parch + t.sibsp).astype(bool)
# astype(bool) convertit le type en booléen
# qui est donc True si la somme des freres/soeurs/conjoint et enfants/parents est 0

Le *deck* (étage du navire) est indiqué par la première lettre de l'identifiant de cabine (par exemple la C85 est au deck C).  
On utilise la fonction *lambda* pour récupérer la valeur de l'objet appelant (ici *s* est la première lettre de l'identifiant cabine) :

In [ ]:
t['deck'] = t.cabin.str[0].map(lambda s: np.nan if s == "T" else s)

On veut maintenant créer une colonne indiquant si un passager est un enfant, une femme ou un homme.  
On commence par écrire une fonction en python :

In [ ]:
def woman_child_or_man(passenger):
    age, sex = passenger
    if age < 16:
        return "child"
    else:
        if sex=="male": return "man"
        else: return "woman"

L'opérateur *apply* permet alors d'appliquer la fonction à toutes les lignes :

In [ ]:
t['qui'] = t[['age', 'sex']].apply(woman_child_or_man, axis=1)

In [ ]:
t.head()

## Graphiques

On veut visualiser le nombre de survivants par sexe :

In [ ]:
nb_survivants_par_sexe = t[survivant].qui.value_counts().sort_index()
nb_survivants_par_sexe

In [ ]:
# Diagramme en "camembert"
plt.pie(nb_survivants_par_sexe, labels = ['enfants','hommes','femmes'], colors = ['green','blue','pink'])

Ou le nombre de survivants par classe sous la forme d'un diagramme à barres :

In [ ]:
nb_survivants_par_classe = t[survivant].pclass.value_counts().sort_index() # sort_index trie par valeurs croissantes
plt.bar(range(3),nb_survivants_par_classe)

On peut superposer les hitogrammes des hommes et des femmes par âge (*alpha=0.5* pour une transparence à 50%)

In [ ]:
plt.hist(t[hommes].age, range=(0,80), bins=20, color='blue', alpha=0.5)
plt.hist(t[femmes].age, range=(0,80), bins=20, color='red', alpha=0.5)

**seaborn** permet des graphiques  plus sophistiqués

Par exemple le graphique de la répartition des passagers par sexe et par classe :

In [ ]:
sns.catplot("classe", data=t, hue="qui", kind="count")

ou, avec une autre écriture, le graphique de la répartition des survivants par sexes et par classe :

In [ ]:

sns.catplot(x="qui", y="survived", hue="classe", data=t, kind="bar")

Les **diagrammes en boîte** (ou **boîtes à moustaches** ou **box plot**) résument quelques caractéristiques de position du caractère étudié (médiane, quartiles, minimum, maximum ou déciles). Ce diagramme est utilisé principalement pour comparer un même caractère dans deux populations de tailles différentes. Il s'agit de tracer un rectangle allant du premier quartile au troisième quartile et coupé par la médiane. On ajoute alors des segments aux extrémités menant jusqu'aux valeurs extrêmes.  
Par exemple pour la répartion de l'âge des passagers selon la classe et le sexe :

In [ ]:
sns.boxplot(x="classe", y="age", hue="qui", data=t)

Les **violins plots** sont similaires aux box plots, excepté qu’ils permettent de montrer la courbe de densité de probabilité des différentes valeurs. Typiquement, les violins plots présentent un marqueur pour la médiane des données et l’écart interquartile, comme dans un box plot standard.

In [ ]:
plt.figure(figsize=(12,5))
sns.violinplot(x="classe", y="age", hue="qui", data=t)

Une variante de présentation, pour le graphique de répartition du prix du ticket selon la classe :

In [ ]:
plt.figure(figsize=(12,5))
sns.violinplot(x="classe", y="fare", hue="sex", data=t, split=True)

*FacetGrid* permet de superposer des graphiques selon une ou plusieurs caractéristiques. On crée une structure avec *FacetGrid*, et on trace ensuite les graphiques avec *map*

In [ ]:
fig = sns.FacetGrid(t, hue="classe", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "age", shade=True)
fig.add_legend()
fig.set(xlim=(0, 80))

**Note :** Les couleurs par défaut sont souvent peu esthétiques ... On peut utiliser des palettes de couleurs, prédéfinies ou non ("Set1", "Set2", "Set3", "Blues", "Colorblind", ...).  
Cf par exemple :  
http://seaborn.pydata.org/tutorial/color_palettes.html  
https://chrisalbon.com/python/seaborn_color_palettes.html  

On peut faire également un tableau de graphique, par exemple afficher les distributions par âge, selon le sexe et la classe :

In [ ]:
fg = sns.FacetGrid(titanic, col="sex", row="pclass", hue="sex", height=2.5, aspect=2.5, palette="PiYG")
fg.map(sns.kdeplot, "age")
fg.set(xlim=(0, 80));

On peut aussi tracer des courbes de régression, pour évaluer par exemple la probabilité de survie :

In [ ]:
sns.lmplot(x='fare', y='survived', hue="qui", data=t, height=7, logistic=True)

On voit que la probabilité de survie des femmes augmente rapidement avec le prix du ticket, ce qui n'est pas le cas de hommes. Le prix a peu d'influence sur la survie des enfants

Pour plus de détails sur les grilles de graphiques :  
http://seaborn.pydata.org/tutorial/axis_grids.html